# Objective: I explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Step 1
Determine the percentage of the documents in `spam_data` that are spam.

*This function returns a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def step_one():
    numofspam = len(spam_data[spam_data['target']==1])
    totalnum = len(spam_data)
    answer = (numofspam/totalnum)*100
    return answer

In [4]:
step_one()

13.406317300789663

### Step 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

Determine the longest token in the vocabulary.

*This function returns a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def step_two():
    ##a count vectorizer does the following: Convert a collection of text documents to a matrix of token counts
    #create the counter vectorizer
    vectorizer = CountVectorizer()

    #Learn a vocabulary dictionary of all tokens in the raw documents.
    vect = vectorizer.fit(X_train)

    #find the vocabulary words in the now trained vectorizer object with the longest length
    answer = max(vect.get_feature_names(), key=lambda token:len(token))

    return answer

In [6]:
step_two()

'com1win150ppmx3age16subscription'

### Step 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function  returns the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def step_three():
    #create and train a countvectorizer
    vectorizer = CountVectorizer()
    #Learn a vocabulary dictionary of all tokens in the raw documents.
    vect = vectorizer.fit(X_train)

    #Transform documents to document-term matrix.
    X_train_vectorized = vect.transform(X_train)

    #create a multinomial Naive Bayes Classifier with alpha = 0.1
    clf = MultinomialNB(alpha=0.1)

    #train the multinomial Naive Bayes Classifier, using the document-term matrix as training data
    clf.fit(X_train_vectorized, y_train)

    #using the created classifier predict results
    ##create the document-term matrix of the test data to put into the trained NB classifier
    X_test_vectorized = vect.transform(X_test)
    predictions = clf.predict(X_test_vectorized)

    #get the AUC score
    ## AUC is the area under the ROC curve
    ### ROC score tells us about how good the model can distinguish between two things
    ### AUC closer to 1 means a good job of distinguishing the positive and the negative values
    answer = roc_auc_score(y_test, predictions)
    return answer

In [8]:
step_three()

0.97208121827411165

### Step 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

Determine the 20 features that have the smallest tf-idf and the 20 that have the largest tf-idf.

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series  be the feature name, and the data  be the tf-idf.

The series of 20 features with smallest tf-idfs is sorted by smallest tfidf first, the list of 20 features with largest tf-idfs is sorted largest first. 

*This functions returns a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def step_four():
    #train the TfidVectorizer
    ##tfidVectorizer is similar to countvectorizer except with TFIDFVectorizer 
    ##the value increases proportionally to count, but is offset by the 
    ##frequency of the word in the corpus. - This is the IDF (inverse document frequency part).
    vect = TfidfVectorizer().fit(X_train)

    #Transform documents to document-term matrix.
    X_train_vectorized = vect.transform(X_train)

    #get the vocabulary from the TfidVectorizer and store in an numpy array
    feature_names = np.array(vect.get_feature_names())

    #for each feature, get the maximum tfidf_index and then return a sorted array(but with indexes and in ascending order)
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

    #get the features that correspond with the first 20 elements of the sorted index(and therefore the 20 features with the
    ##smallest Tfid values)
    small_index = feature_names[sorted_tfidf_index[:20]]

    #get the tfid index value for those 20 features
    small_value = X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[:20]]

    #create a tuple combining the feature with it's Tfid value
    smallTuples = [(value, word) for word, value in zip(small_index, small_value)]

    #sort the tuples
    smallTuples.sort()

    #create two seperate arrays to store the index of the smallest values and the actual value of the smallest values
    small_index = [element[1] for element in smallTuples]
    small_value = [element[0] for element in smallTuples]

    #create a pandas df with the two arrays just created
    small_series = pd.Series(small_value,index=small_index)

    #Determine the features with the biggest Tfidf values
    big_index = feature_names[sorted_tfidf_index[-20:]]
    big_value = X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[-20:]]

    #set value as negative so that sort() sorts it correctly
    ##.sort() sorts in ascending order, but we want descending order
    bigTuples = [(-value,word) for word,value in zip(big_index, big_value)]
    bigTuples.sort()
    big_index = [element[1] for element in bigTuples]

    #apply another negative so that that value is converted back to positive
    big_value = [-element[0] for element in bigTuples]
    big_series = pd.Series(big_value, index = big_index)

    #merge the two df together to create the solution
    a = (small_series, big_series)
    return a

In [10]:
step_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Step 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function returns the AUC score as a float.*

In [11]:
def step_five():
    #fit the training data using the Tfidf Vectorizer ignoring terms that have a document frequency lower than 3
    ## Document frequency is the number of times a given term or query appears in a specific document
    vect = TfidfVectorizer(min_df=3).fit(X_train)

    #transform the training data
    X_train_vectorized = vect.transform(X_train)

    #create a multinomial NB with alpha = 0.1
    model = MultinomialNB(alpha=0.1)

    #fit a multinomial NB with alpha = 0.1 with the transformed train data
    model.fit(X_train_vectorized, y_train)

    #predict labels using the trained NB classifier, using the transformed test data
    predictions = model.predict(vect.transform(X_test))

    #return AUC score
    answer = roc_auc_score(y_test, predictions)
    return answer

In [12]:
step_five()

0.94162436548223349

### Step 6

Determine the average length of documents (number of characters) for not spam and spam documents?

*This function returns a tuple (average length not spam, average length spam).*

In [13]:
def step_six():
    #create a column in the spam_data df that is just the # of characters
    spam_data['length'] = spam_data['text'].str.len()

    #create a nonSpam df from the spam_data df 
    nonSpam = spam_data[spam_data['target'] == 0]

    #create a spam df from the spam_data df
    spam = spam_data[spam_data['target'] == 1]

    #calculate the average length of non spam text
    num_of_nonspam = nonSpam['length'].sum()
    totalnum_of_nonspam = len(nonSpam)

    #calculate the average lenght of spam text
    num_of_spam = spam['length'].sum()
    totalnum_of_spam = len(spam)

    answer = (num_of_nonspam/totalnum_of_nonspam, num_of_spam/totalnum_of_spam)
    
    return answer

In [14]:
step_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
This function helps combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Step 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix add an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function returns the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def step_seven():
    from sklearn.svm import SVC
    #create a Tfidf Vectorizer that ignores terms that have document frequency lower than 5 and train it off training data
    vect = TfidfVectorizer(min_df=5).fit(X_train)

    #transform the vectorizer into a document matrix
    X_train_vectorized = vect.transform(X_train)

    #get the length of each text in the test data
    a=X_train.str.len()

    #add feature to the document matrix
    X_train_vectorized = add_feature(X_train_vectorized, a)

    #create a document matrix of test data
    X_test_vectorized = vect.transform(X_test)

    #get the length of each text in the test data
    b=X_test.str.len()

    #add the feature to the test document matrix
    X_test_vectorized = add_feature(X_test_vectorized, b)

    #create the support vector machine model
    model = SVC(C=10000)

    #train the support vector machine model
    model.fit(X_train_vectorized, y_train)

    #get the predicted labels of the support vector machine base of the document matrix
    predictions = model.predict(X_test_vectorized) 

    #get the AUC score
    answer = roc_auc_score(y_test, predictions)
    
    return answer

In [17]:
step_seven()

0.95813668234215565

### Step 8

Determine the average number of digits per document for not spam and spam documents?

*This function returns a tuple (average # digits not spam, average # digits spam).*

In [18]:
def step_eight():
    #import the regular expression package
    import re

    #go through all the spam text and find all the digits
    spam = [re.findall("\d",i) for i in spam_data['text'][spam_data.target==1]]

    #go through all the non spam text and find all the digits
    non_spam = [re.findall("\d",i) for i in spam_data['text'][spam_data.target==0]]

    #apply the len function to all digits to get the number of digits in nonspam 
    #and convert to a list that numpy can calculate the mean of
    avg_num_digits_nonspam = np.mean(list(map(len,non_spam)))

    #apply the len function to all digits to get the number of digits in spam 
    #and convert to a list that numpy can calculate the mean of
    avg_num_digits_spam = np.mean(list(map(len,spam)))                   

    answer = (avg_num_digits_nonspam,avg_num_digits_spam)
    return answer

In [19]:
step_eight()

(0.29927461139896372, 15.759036144578314)

### Step 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function returns the AUC score as a float.*

In [26]:
from sklearn.linear_model import LogisticRegression

def step_nine():
    from sklearn.linear_model import LogisticRegression
    import re

    #Fit a Tfidf Vectorizer ignoring document frequency lower 5 and using n-grams from n=1 to n=3
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)

    #transform Tfidf Vectorizer for X_train
    X_train_vectorized = vect.transform(X_train)

    #add training text length
    a = X_train.str.len()
    X_train_vectorized = add_feature(X_train_vectorized, a)

    #in training data find all digits
    X_train_digits = X_train.str.findall(r'(\d)')

    #add number of digits in training text as a feature
    num_of_digits = list(map(len, X_train_digits))
    X_train_vectorized = add_feature(X_train_vectorized, num_of_digits)

    #transform Tfidf Vectorizer for X_test
    X_test_vectorized = vect.transform(X_test)

    #add test text length
    b=X_test.str.len()
    X_test_vectorized = add_feature(X_test_vectorized, b)

    #add number of digits in test text as a feature
    X_test_digits = X_test.str.findall(r'(\d)')
    num_of_digits = list(map(len, X_test_digits))
    X_test_vectorized = add_feature(X_test_vectorized,num_of_digits )

    #create logistic regression classifier
    model = LogisticRegression(C=100)

    #fit logistic regression classifier with document matrix train data with added features
    model.fit(X_train_vectorized, y_train)

    #get predicted labels
    predictions = model.predict(X_test_vectorized)

    #get the ROC
    answer = roc_auc_score(y_test, predictions)
    return answer

In [27]:
step_nine()

0.96533283533945646

### Step 10

Determine the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents. Use `\w` and `\W` character classes

*This function returns a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def step_ten():
    import re
    #find all the words in the spam data
    spam = [re.findall("\W",i) for i in spam_data['text'][spam_data.target==1]]
    
    #find all the words in the non spam data
    non_spam = [re.findall("\W",i) for i in spam_data['text'][spam_data.target==0]]

    #calculate the average number of nonspam words
    avg_num_of_nonspam_words = np.mean(list(map(len,non_spam)))
    avg_num_of_spam_words = np.mean(list(map(len,spam)))
    answer = (avg_num_of_nonspam_words,avg_num_of_spam_words)
    return answer

In [23]:
step_ten()

(17.291813471502589, 29.041499330655956)

### Step 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This  make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients  be sorted smallest first, the list of 10 largest coefficients  be sorted largest first.

The three features that were added to the document term matrix  have the following names  they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function returns a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
def step_eleven():
    #train Count Vectorizer ignoring terms with document frequency lower than 5 and using n-grams from n=2 to n=5
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)

    #transform to document matrix
    X_train_vectorized = vect.transform(X_train)

    #add feature that is length of text
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.len())

    #add feature that is number of digits
    X_train_digits = X_train.str.findall(r'(\d)')
    X_train_vectorized = add_feature(X_train_vectorized, list(map(len, X_train_digits)))

    #add feature that is number of words
    X_train_nonChar = X_train.str.findall(r'(\W)')
    X_train_vectorized = add_feature(X_train_vectorized, list(map(len, X_train_nonChar)))

    #create document matrix of text data
    X_test_vectorized = vect.transform(X_test)

    #add feature that is length of text
    X_test_vectorized = add_feature(X_test_vectorized, X_test.str.len())

    #add feature that is number of digits 
    X_test_digits = X_test.str.findall(r'(\d)')
    X_test_vectorized = add_feature(X_test_vectorized, list(map(len, X_test_digits)))

    #add feature that is number of words
    X_test_nonChar = X_test.str.findall(r'(\W)')
    X_test_vectorized = add_feature(X_test_vectorized, list(map(len, X_test_nonChar)))

    #create logistic regression classifier
    model = LogisticRegression(C=100)

    #train the logistic regression classifier
    model.fit(X_train_vectorized, y_train)

    #create prediction 
    predictions = model.predict(X_test_vectorized)


    #find the 10 smallest and 10 largest coefficients from the model
    #get all the feature names
    feature_names = np.array(vect.get_feature_names())

    #add to the list of feature names the 'length_of_doc', 'digit_count', 'non_word_char_count'
    ##these are features that were later added
    feature_names = np.concatenate((feature_names, np.array(['length_of_doc', 'digit_count', 'non_word_char_count'])))

    #get a sorted array(by index) of logistic coefficients
    sorted_coef_index = model.coef_[0].argsort()

    #the list of smallest coefficients by index in ascending order
    small_coefficient = list(feature_names[sorted_coef_index[:10]])

    #the list of largest coefficients by index in descending order
    large_coefficient = list(feature_names[sorted_coef_index[:-11:-1]])

    #get AUC score
    auc_score = roc_auc_score(y_test, predictions)

    #create a tuple that is the answer
    answer = (auc_score,small_coefficient,large_coefficient)
    
    return answer

In [25]:
step_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])